In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_cleaning.create_staging_tables import create_staging_bill_df_STEP_THREE

In [2]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [5]:
raw_bill_df = pd.read_sql_query('select * from "bill_api"', con=engine)
raw_sponsor_df = pd.read_sql_query('select * from "sponsor_api"', con=engine)
raw_vote_df = pd.read_sql_query('select * from "vote_api"', con=engine)

In [14]:
bill_df = create_staging_bill_df_STEP_THREE(raw_bill_df, raw_sponsor_df)

/Users/emilykarboski/galvanize/washington_state_legislation/data_cleaning/create_staging_tables.py:372: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sponsor_df_reformatted['bill_num'] = sponsor_df_reformatted['bill_id'].apply(lambda x: x.split()[1])
/Users/emilykarboski/galvanize/washington_state_legislation/data_cleaning/create_staging_tables.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sponsor_df_reformatted['bill_num_unique'] = sponsor_df_reformatted['biennium'] + ' ' + sponsor_df_reformatted['bill_num']


In [7]:
raw_vote_df['bill_unique'] = raw_vote_df['biennium'] + ' ' + raw_vote_df['bill_id']

In [10]:
bills_already_voted_on = raw_vote_df['bill_unique'].unique()

In [19]:
def determine_if_reached_vote(bill_unique):
    if bill_unique in bills_already_voted_on:
        return True
    return False

In [20]:
bill_df['reached_vote'] = bill_df['bill_unique'].apply(determine_if_reached_vote)

In [21]:
bill_df

,biennium,bill_id,class,description,htm_create_date,htm_last_modified_date,htm_url,long_friendly_name,name,type,bill_unique,bill_num,bill_num_unique,sponsor_agency,primary_sponsor_id,secondary_sponsors,reached_vote
0,1991-92,HB 1001,Bills,None,1991-08-30T00:00:00,2006-07-10T17:13:53.543,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1001,1001,House Bills,1991-92 HB 1001,1001,1991-92 1001,House,251,"[444, 72, 34, 54, 48, 7, 481, 301, 478, 264, 2...",False
1,1991-92,SHB 1001,Bills,None,1991-02-01T00:00:00,2006-07-10T17:13:54.903,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1001,1001-S,House Bills,1991-92 SHB 1001,1001,1991-92 1001,House,251,"[444, 72, 34, 54, 48, 7, 481, 301, 478, 264, 2...",True
2,1991-92,HB 1002,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.637,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1002,1002,House Bills,1991-92 HB 1002,1002,1991-92 1002,House,251,"[332, 474, 475, 219, 285, 227, 180, 110, 484, ...",False
3,1991-92,HB 1003,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.747,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1003,1003,House Bills,1991-92 HB 1003,1003,1991-92 1003,House,311,"[474, 54, 110]",False
4,1991-92,SHB 1003,Bills,None,1991-02-21T00:00:00,2006-07-10T17:14:07.357,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1003,1003-S,House Bills,1991-92 SHB 1003,1003,1991-92 1003,House,311,"[474, 54, 110]",True
5,1991-92,HB 1004,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.53,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1004,1004,House Bills,1991-92 HB 1004,1004,1991-92 1004,House,311,"[475, 304, 110, 32, 54]",False
6,1991-92,HB 1005,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:12.747,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1005,1005,House Bills,1991-92 HB 1005,1005,1991-92 1005,House,311,"[23, 475]",False
7,1991-92,HB 1006,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.857,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1006,1006,House Bills,1991-92 HB 1006,1006,1991-92 1006,House,311,[110],False
8,1991-92,HB 1007,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:12.373,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1007,1007,House Bills,1991-92 HB 1007,1007,1991-92 1007,House,311,"[475, 276, 285, 110, 180, 48, 443, 472]",False
9,1991-92,HB 1008,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:12.183,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1008,1008,House Bills,1991-92 HB 1008,1008,1991-92 1008,House,311,"[474, 475, 110]",False


In [22]:
def filter_out_old_versions(row):
    count = len(bill_df[bill_df['bill_num_unique'] == row['bill_num_unique']])
    if count > 1 and row['reached_vote'] == False:
        return False
    return True

In [23]:
bill_df['is_final_version'] = bill_df.apply(filter_out_old_versions, axis=1)

In [26]:
bill_df = bill_df[bill_df['is_final_version'] == True]

In [30]:
bill_df = bill_df.drop('is_final_version', axis=1)

In [31]:
bill_df

,biennium,bill_id,class,description,htm_create_date,htm_last_modified_date,htm_url,long_friendly_name,name,type,bill_unique,bill_num,bill_num_unique,sponsor_agency,primary_sponsor_id,secondary_sponsors,reached_vote
1,1991-92,SHB 1001,Bills,None,1991-02-01T00:00:00,2006-07-10T17:13:54.903,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1001,1001-S,House Bills,1991-92 SHB 1001,1001,1991-92 1001,House,251,"[444, 72, 34, 54, 48, 7, 481, 301, 478, 264, 2...",True
2,1991-92,HB 1002,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.637,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1002,1002,House Bills,1991-92 HB 1002,1002,1991-92 1002,House,251,"[332, 474, 475, 219, 285, 227, 180, 110, 484, ...",False
4,1991-92,SHB 1003,Bills,None,1991-02-21T00:00:00,2006-07-10T17:14:07.357,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1003,1003-S,House Bills,1991-92 SHB 1003,1003,1991-92 1003,House,311,"[474, 54, 110]",True
5,1991-92,HB 1004,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.53,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1004,1004,House Bills,1991-92 HB 1004,1004,1991-92 1004,House,311,"[475, 304, 110, 32, 54]",False
6,1991-92,HB 1005,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:12.747,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1005,1005,House Bills,1991-92 HB 1005,1005,1991-92 1005,House,311,"[23, 475]",False
7,1991-92,HB 1006,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.857,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1006,1006,House Bills,1991-92 HB 1006,1006,1991-92 1006,House,311,[110],False
8,1991-92,HB 1007,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:12.373,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1007,1007,House Bills,1991-92 HB 1007,1007,1991-92 1007,House,311,"[475, 276, 285, 110, 180, 48, 443, 472]",False
10,1991-92,SHB 1008,Bills,None,1991-02-21T00:00:00,2006-07-10T17:14:07.463,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1008,1008-S,House Bills,1991-92 SHB 1008,1008,1991-92 1008,House,311,"[474, 475, 110]",True
11,1991-92,SHB 1008,Bills,4-10-91 ENROLLED BILL,1991-04-10T00:00:00,2006-07-10T17:50:25.86,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1008 - as passed by the ...,1008-S.PL,House Passed Legislature,1991-92 SHB 1008,1008,1991-92 1008,House,311,"[474, 475, 110]",True
12,1991-92,SHB 1008,Bills,None,1991-05-06T00:00:00,2012-08-12T14:12:10.747,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1008 - Session Law,1008-S.SL,Session Laws,1991-92 SHB 1008,1008,1991-92 1008,House,311,"[474, 475, 110]",True
